In [1]:
%load_ext autoreload
%autoreload 2

In [19]:
import os 
import sys
import time

project_dir_path = '/home/onoue/ws/lukasiewicz_1'
sys.path.append(project_dir_path)

print(sys.path)


from src.setup_problem_loss_custom_obj import Setup
from src.misc import visualize_result

import cvxpy as cp
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

from src.misc import *

['/home/onoue/ws/lukasiewicz_1/notebooks/pima_indian_diabetes', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/home/onoue/ws/lukasiewicz_1/myenv/lib/python3.10/site-packages', '/home/onoue/ws/lukasiewicz_1', '/home/onoue/ws/lukasiewicz_1', '/home/onoue/ws/lukasiewicz_1', '/home/onoue/ws/lukasiewicz_1', '/home/onoue/ws/lukasiewicz_1']


In [9]:
# from src.setup_problem_loss_tmp import Setup
from src.setup_problem_loss_custom_obj import Setup

data_dir_path = os.path.join(project_dir_path, 'inputs/pima_indian_diabetes')
train_data_dir_path = os.path.join(data_dir_path, "train_4")
file_list = os.listdir(train_data_dir_path)

L_files = [filename.split('.csv')[0] for filename in file_list 
           if filename.startswith('L') and filename.endswith('.csv')]

U_files = [filename.split('.csv')[0] for filename in file_list 
           if filename.startswith('U') and filename.endswith('.csv')]

file_names_dict = {
    'supervised': L_files,
    'unsupervised': U_files,
    'rule': ['rules_4']
}

In [64]:
def construct_objective_function(obj, c1, c2_dict):
    """
    目的関数を構成する．
    c1 は logical constraints を，
    c2 は consistency constraints を
    満足する度合いを表す．
    """

    function = 0

    for j in range(obj.len_j):
        w = obj.w_j[j]
        function += 1/2 * (cp.norm2(w) ** 2)

    # for j in range(obj.len_j):
    #     for l in range(obj.len_l):
    #         xi = obj.xi_jl[j, l]
    #         function += c1 * xi

    # 交差エントロピー
    for p_name, p in obj.predicates_dict.items():
        x = obj.L[p_name][:, :-1]
        y = obj.L[p_name][:, -1]
        y_pred = p(x)
        value = log_loss(y, y_pred)
        function += c1 * value

    print(f'c1: {c1}')

    # for h in range(obj.len_h):
    #     xi = obj.xi_h[h, 0]
    #     function += c2 * xi

    df = obj.KB_info
    groups = set(df['group'].to_list())

    if groups != set(c2_dict.keys()):
        raise ValueError("The keys of c2_dict do not match the groups.")

    for group in groups:
        target_index = df[df['group'] == group].index.tolist()

        xi = obj.xi_h[target_index]
        c2_weighted = c2_dict[group]
        function += c2_weighted * cp.norm(xi)

        print(f'c2: {c2_weighted}')

    obj.objective_function = cp.Minimize(function)

    return obj.objective_function


In [80]:
def construct_objective_function_2(obj, c1, c2_dict):
    """
    目的関数を構成する．
    c1 は logical constraints を，
    c2 は consistency constraints を
    満足する度合いを表す．
    """

    function = 0

    for j in range(obj.len_j):
        w = obj.w_j[j]
        function += 1/2 * (cp.norm2(w) ** 2)

    # for j in range(obj.len_j):
    #     for l in range(obj.len_l):
    #         xi = obj.xi_jl[j, l]
    #         function += c1 * xi

    # 交差エントロピー
    for p_name, p in obj.predicates_dict.items():
        x = obj.L[p_name][:, :-1]
        y = obj.L[p_name][:, -1]
        y_pred = p(x)
        value = log_loss(y, y_pred)
        function += c1 * value

    print(f'c1: {c1}')

    # for h in range(obj.len_h):
    #     xi = obj.xi_h[h, 0]
    #     function += c2 * xi

    df = obj.KB_info
    groups = set(df['group'].to_list())
    shap_vals = df['shap']

    if groups != set(c2_dict.keys()):
        raise ValueError("The keys of c2_dict do not match the groups.")

    for group in groups:
        target_index = df[df['group'] == group].index.tolist()

        xi = obj.xi_h[target_index]
        shap = df.loc[target_index, 'shap'].values.reshape(1, -1)
        """
        shap が norm を計算する際に二乗されるのが良くない
        """
        xi = cp.multiply(xi, shap)

        c2_weighted = c2_dict[group]
        function += c2_weighted * cp.norm(xi)

        print(f'c2: {c2_weighted}')

    obj.objective_function = cp.Minimize(function)

    return obj.objective_function


In [84]:
def construct_objective_function_3(obj, c1, c2_dict):
    """
    目的関数を構成する．
    c1 は logical constraints を，
    c2 は consistency constraints を
    満足する度合いを表す．
    """

    function = 0

    for j in range(obj.len_j):
        w = obj.w_j[j]
        function += 1/2 * (cp.norm2(w) ** 2)

    # for j in range(obj.len_j):
    #     for l in range(obj.len_l):
    #         xi = obj.xi_jl[j, l]
    #         function += c1 * xi

    # 交差エントロピー
    for p_name, p in obj.predicates_dict.items():
        x = obj.L[p_name][:, :-1]
        y = obj.L[p_name][:, -1]
        y_pred = p(x)
        value = log_loss(y, y_pred)
        function += c1 * value

    print(f'c1: {c1}')

    # for h in range(obj.len_h):
    #     xi = obj.xi_h[h, 0]
    #     function += c2 * xi

    df = obj.KB_info
    groups = set(df['group'].to_list())
    shap_vals = df['shap']

    if groups != set(c2_dict.keys()):
        raise ValueError("The keys of c2_dict do not match the groups.")

    for group in groups:
        target_index = df[df['group'] == group].index.tolist()

        xi = obj.xi_h[target_index]
        shap = df.loc[target_index, 'shap'].values.reshape(1, -1)
        shap = np.sqrt(shap)

        xi = cp.multiply(xi, shap)

        c2_weighted = c2_dict[group]
        function += c2_weighted * cp.norm(xi)

        print(f'c2: {c2_weighted}')

    obj.objective_function = cp.Minimize(function)

    return obj.objective_function


In [94]:
def construct_objective_function_4(obj, c1):
    """
    目的関数を構成する．
    c1 は logical constraints を，
    c2 は consistency constraints を
    満足する度合いを表す．
    """

    function = 0

    for j in range(obj.len_j):
        w = obj.w_j[j]
        function += 1/2 * (cp.norm2(w) ** 2)

    # for j in range(obj.len_j):
    #     for l in range(obj.len_l):
    #         xi = obj.xi_jl[j, l]
    #         function += c1 * xi

    # 交差エントロピー
    for p_name, p in obj.predicates_dict.items():
        x = obj.L[p_name][:, :-1]
        y = obj.L[p_name][:, -1]
        y_pred = p(x)
        value = log_loss(y, y_pred)
        function += c1 * value

    print(f'c1: {c1}')

    # for h in range(obj.len_h):
    #     xi = obj.xi_h[h, 0]
    #     function += c2 * xi

    df = obj.KB_info
    groups = set(df['group'].to_list())

    for group in groups:
        target_index = df[df['group'] == group].index.tolist()

        xi = obj.xi_h[target_index]
        shap_partial_sum = df.loc[target_index, 'shap'].sum()

        c2_weighted = shap_partial_sum
        function += c2_weighted * cp.norm(xi)

        print(f'c2: {c2_weighted}')

    obj.objective_function = cp.Minimize(function)

    return obj.objective_function

In [98]:
def construct_objective_function_5(obj, c1, c2):
    """
    目的関数を構成する．
    c1 は logical constraints を，
    c2 は consistency constraints を
    満足する度合いを表す．
    """

    function = 0

    for j in range(obj.len_j):
        w = obj.w_j[j]
        function += 1/2 * (cp.norm2(w) ** 2)

    # for j in range(obj.len_j):
    #     for l in range(obj.len_l):
    #         xi = obj.xi_jl[j, l]
    #         function += c1 * xi

    # 交差エントロピー
    for p_name, p in obj.predicates_dict.items():
        x = obj.L[p_name][:, :-1]
        y = obj.L[p_name][:, -1]
        y_pred = p(x)
        value = log_loss(y, y_pred)
        function += c1 * value

    print(f'c1: {c1}')

    # for h in range(obj.len_h):
    #     xi = obj.xi_h[h, 0]
    #     function += c2 * xi


    df = obj.KB_info
    groups = set(df['group'].to_list())
    shap_total_sum = df['shap'].sum()

    for group in groups:
        target_index = df[df['group'] == group].index.tolist()

        xi = obj.xi_h[target_index]
        shap_partial_sum = df.loc[target_index, 'shap'].sum()

        shap_ratio = shap_partial_sum / shap_total_sum

        c2_weighted = shap_ratio * c2
        function += c2_weighted * cp.norm(xi)

        print(f'c2: {c2_weighted}')

    obj.objective_function = cp.Minimize(function)

    return obj.objective_function

In [102]:
def construct_objective_function_6(obj, c1, c2):
    """
    目的関数を構成する．
    c1 は logical constraints を，
    c2 は consistency constraints を
    満足する度合いを表す．
    """

    function = 0

    for j in range(obj.len_j):
        w = obj.w_j[j]
        function += 1/2 * (cp.norm2(w) ** 2)


    # 交差エントロピー
    for p_name, p in obj.predicates_dict.items():
        x = obj.L[p_name][:, :-1]
        y = obj.L[p_name][:, -1]
        y_pred = p(x)
        value = log_loss(y, y_pred)
        function += c1 * value

    print(f'c1: {c1}')

    df = obj.KB_info
    groups = set(df['group'].to_list())
    shap_partial_means = []
    target_index_list = []

    for group in groups:
        target_index = df[df['group'] == group].index.tolist()
        target_index_list.append(target_index)

        shap_partial_mean = df.loc[target_index, 'shap'].mean()
        shap_partial_means.append(shap_partial_mean)
    
    shap_partial_means_sum = sum(shap_partial_means)

    for target_index, shap_partial_mean in zip(target_index_list, shap_partial_means):
        xi = obj.xi_h[target_index]
        shap_ratio = shap_partial_mean / shap_partial_means_sum

        c2_weighted = shap_ratio * c2
        function += c2_weighted * cp.norm(xi)

        print(f'c2: {c2_weighted}')

    obj.objective_function = cp.Minimize(function)

    return obj.objective_function

In [103]:
from src.setup_problem_loss_custom_obj import Setup

data_dir_path = os.path.join(project_dir_path, 'inputs/pima_indian_diabetes')
train_data_dir_path = os.path.join(data_dir_path, "train_4")
file_list = os.listdir(train_data_dir_path)

L_files = [filename.split('.csv')[0] for filename in file_list 
           if filename.startswith('L') and filename.endswith('.csv')]

U_files = [filename.split('.csv')[0] for filename in file_list 
           if filename.startswith('U') and filename.endswith('.csv')]

file_names_dict = {
    'supervised': L_files,
    'unsupervised': U_files,
    'rule': ['rules_4']
}

# c2_dict = {1: 10, 2: 2.5}

obj_constructor = construct_objective_function_6
problem_instance = Setup(train_data_dir_path, file_names_dict, obj_constructor)
objective, constraints = problem_instance.main(c1=10, c2=10)

start_time = time.time()
problem = cp.Problem(objective, constraints)
result = problem.solve(verbose=True)
end_time = time.time()
print()
print(f'学習時間: {end_time - start_time} 秒')


print()
print()
print()


test_data_dir_path = os.path.join(data_dir_path, "test_4")
file_path = os.path.join(test_data_dir_path, "L_Outcome.csv")
test_df = pd.read_csv(file_path, index_col=0)
display(test_df.head())

test_data = {
    'Outcome': np.array(test_df)
}

p_dict = problem_instance.predicates_dict
selected_predicates = ['Outcome']
selected_p_dict = {key: value for key, value in p_dict.items() if key in selected_predicates}

X_test = test_data['Outcome'][:, :-1]
y_test = test_data['Outcome'][:, -1]

y_pred = p_dict['Outcome'](X_test).value

y_pred_interpreted = np.where(y_pred >= 0.5, 1, -1)

print("Outcome's coefficients: ")
print(problem_instance.w_j.value[3])
print()

accuracy = accuracy_score(y_test, y_pred_interpreted)
print("Accuracy: ")
print(accuracy)
print()

conf_matrix = confusion_matrix(y_test, y_pred_interpreted)
print("Confusion matrix: ")
print(conf_matrix)
print()

class_report = classification_report(y_test, y_pred_interpreted)
print("classification report: ")
print(class_report)
print()

roc_auc = roc_auc_score(y_test, y_pred)
print("auc: ")
print(roc_auc)

Loading data ...
Done in 0.1521587371826172 seconds! 

Loading rules ...
Done in 0.0031485557556152344 seconds! 

Identifying predicates ...
Done in 0.0023131370544433594 seconds! 

Constructing objective function ...
c1: 10
c2: 4.9501482108448265
c2: 5.0498517891551735
Done in 0.05614876747131348 seconds! 

Constructing constraints ...
Done in 10.774420976638794 seconds! 

All done
                                     CVXPY                                     
                                     v1.3.2                                    
(CVXPY) Dec 03 04:42:44 PM: Your problem has 220 variables, 26224 constraints, and 0 parameters.
(CVXPY) Dec 03 04:42:49 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Dec 03 04:42:49 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Dec 03 04:42:49 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------

/home/onoue/ws/lukasiewicz_1/myenv/lib/python3.10/site-packages/cvxpy/problems/problem.py:1387: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
/home/onoue/ws/lukasiewicz_1/myenv/lib/python3.10/site-packages/cvxpy/atoms/elementwise/log.py:35: RuntimeWarning: invalid value encountered in log
  return np.log(values[0])


,Pregnancies,Glucose,BloodPressure,SkinThickness,BMI,DiabetesPedigreeFunction,Age,target
419,3,129,64,29,26.4,0.219,28,1
186,8,181,68,36,30.1,0.615,60,1
556,1,97,70,40,38.1,0.218,30,-1
738,2,99,60,17,36.6,0.453,21,-1
320,4,129,60,12,27.5,0.527,31,-1


Outcome's coefficients: 
[ 2.36457900e-02  4.54651335e-03 -7.26440381e-03  3.36881882e-05
  3.82228997e-03  1.16155601e-01  1.76710899e-03  5.46360409e-03]

Accuracy: 
0.7611940298507462

Confusion matrix: 
[[79 11]
 [21 23]]

classification report: 
              precision    recall  f1-score   support

        -1.0       0.79      0.88      0.83        90
         1.0       0.68      0.52      0.59        44

    accuracy                           0.76       134
   macro avg       0.73      0.70      0.71       134
weighted avg       0.75      0.76      0.75       134


auc: 
0.8035353535353535


In [99]:
from src.setup_problem_loss_custom_obj import Setup

data_dir_path = os.path.join(project_dir_path, 'inputs/pima_indian_diabetes')
train_data_dir_path = os.path.join(data_dir_path, "train_4")
file_list = os.listdir(train_data_dir_path)

L_files = [filename.split('.csv')[0] for filename in file_list 
           if filename.startswith('L') and filename.endswith('.csv')]

U_files = [filename.split('.csv')[0] for filename in file_list 
           if filename.startswith('U') and filename.endswith('.csv')]

file_names_dict = {
    'supervised': L_files,
    'unsupervised': U_files,
    'rule': ['rules_4']
}

# c2_dict = {1: 10, 2: 2.5}

obj_constructor = construct_objective_function_5
problem_instance = Setup(train_data_dir_path, file_names_dict, obj_constructor)
objective, constraints = problem_instance.main(c1=10, c2=10)

start_time = time.time()
problem = cp.Problem(objective, constraints)
result = problem.solve(verbose=True)
end_time = time.time()
print()
print(f'学習時間: {end_time - start_time} 秒')


print()
print()
print()


test_data_dir_path = os.path.join(data_dir_path, "test_4")
file_path = os.path.join(test_data_dir_path, "L_Outcome.csv")
test_df = pd.read_csv(file_path, index_col=0)
display(test_df.head())

test_data = {
    'Outcome': np.array(test_df)
}

p_dict = problem_instance.predicates_dict
selected_predicates = ['Outcome']
selected_p_dict = {key: value for key, value in p_dict.items() if key in selected_predicates}

X_test = test_data['Outcome'][:, :-1]
y_test = test_data['Outcome'][:, -1]

y_pred = p_dict['Outcome'](X_test).value

y_pred_interpreted = np.where(y_pred >= 0.5, 1, -1)

print("Outcome's coefficients: ")
print(problem_instance.w_j.value[3])
print()

accuracy = accuracy_score(y_test, y_pred_interpreted)
print("Accuracy: ")
print(accuracy)
print()

conf_matrix = confusion_matrix(y_test, y_pred_interpreted)
print("Confusion matrix: ")
print(conf_matrix)
print()

class_report = classification_report(y_test, y_pred_interpreted)
print("classification report: ")
print(class_report)
print()

roc_auc = roc_auc_score(y_test, y_pred)
print("auc: ")
print(roc_auc)

Loading data ...
Done in 0.14368057250976562 seconds! 

Loading rules ...
Done in 0.0017161369323730469 seconds! 

Identifying predicates ...
Done in 0.0010883808135986328 seconds! 

Constructing objective function ...
c1: 10
c2: 8.382231431965785
c2: 1.617768568034215
Done in 0.02666759490966797 seconds! 

Constructing constraints ...
Done in 11.18724799156189 seconds! 

All done
                                     CVXPY                                     
                                     v1.3.2                                    
(CVXPY) Dec 03 04:31:34 PM: Your problem has 220 variables, 26224 constraints, and 0 parameters.
(CVXPY) Dec 03 04:31:38 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Dec 03 04:31:38 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Dec 03 04:31:38 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
---------------

/home/onoue/ws/lukasiewicz_1/myenv/lib/python3.10/site-packages/cvxpy/atoms/elementwise/log.py:35: RuntimeWarning: invalid value encountered in log
  return np.log(values[0])


,Pregnancies,Glucose,BloodPressure,SkinThickness,BMI,DiabetesPedigreeFunction,Age,target
419,3,129,64,29,26.4,0.219,28,1
186,8,181,68,36,30.1,0.615,60,1
556,1,97,70,40,38.1,0.218,30,-1
738,2,99,60,17,36.6,0.453,21,-1
320,4,129,60,12,27.5,0.527,31,-1


Outcome's coefficients: 
[ 2.36403036e-02  4.54581339e-03 -7.26428322e-03  3.25093892e-05
  3.82077062e-03  1.16458337e-01  1.76843945e-03  5.46333478e-03]

Accuracy: 
0.7611940298507462

Confusion matrix: 
[[79 11]
 [21 23]]

classification report: 
              precision    recall  f1-score   support

        -1.0       0.79      0.88      0.83        90
         1.0       0.68      0.52      0.59        44

    accuracy                           0.76       134
   macro avg       0.73      0.70      0.71       134
weighted avg       0.75      0.76      0.75       134


auc: 
0.8035353535353535


In [85]:
from src.setup_problem_loss_custom_obj import Setup

data_dir_path = os.path.join(project_dir_path, 'inputs/pima_indian_diabetes')
train_data_dir_path = os.path.join(data_dir_path, "train_4")
file_list = os.listdir(train_data_dir_path)

L_files = [filename.split('.csv')[0] for filename in file_list 
           if filename.startswith('L') and filename.endswith('.csv')]

U_files = [filename.split('.csv')[0] for filename in file_list 
           if filename.startswith('U') and filename.endswith('.csv')]

file_names_dict = {
    'supervised': L_files,
    'unsupervised': U_files,
    'rule': ['rules_4']
}

c2_dict = {1: 10, 2: 2.5}

obj_constructor = construct_objective_function_3
problem_instance = Setup(train_data_dir_path, file_names_dict, obj_constructor)
objective, constraints = problem_instance.main(c1=10, c2_dict=c2_dict)

start_time = time.time()
problem = cp.Problem(objective, constraints)
result = problem.solve(verbose=True)
end_time = time.time()
print()
print(f'学習時間: {end_time - start_time} 秒')


print()
print()
print()


test_data_dir_path = os.path.join(data_dir_path, "test_4")
file_path = os.path.join(test_data_dir_path, "L_Outcome.csv")
test_df = pd.read_csv(file_path, index_col=0)
display(test_df.head())

test_data = {
    'Outcome': np.array(test_df)
}

p_dict = problem_instance.predicates_dict
selected_predicates = ['Outcome']
selected_p_dict = {key: value for key, value in p_dict.items() if key in selected_predicates}

X_test = test_data['Outcome'][:, :-1]
y_test = test_data['Outcome'][:, -1]

y_pred = p_dict['Outcome'](X_test).value

y_pred_interpreted = np.where(y_pred >= 0.5, 1, -1)

print("Outcome's coefficients: ")
print(problem_instance.w_j.value[3])
print()

accuracy = accuracy_score(y_test, y_pred_interpreted)
print("Accuracy: ")
print(accuracy)
print()

conf_matrix = confusion_matrix(y_test, y_pred_interpreted)
print("Confusion matrix: ")
print(conf_matrix)
print()

class_report = classification_report(y_test, y_pred_interpreted)
print("classification report: ")
print(class_report)
print()

roc_auc = roc_auc_score(y_test, y_pred)
print("auc: ")
print(roc_auc)

Loading data ...
Done in 0.23522043228149414 seconds! 

Loading rules ...
Done in 0.004548311233520508 seconds! 

Identifying predicates ...
Done in 0.0028524398803710938 seconds! 

Constructing objective function ...
c1: 10
c2: 10
c2: 2.5
Done in 0.03520774841308594 seconds! 

Constructing constraints ...
Done in 14.551660776138306 seconds! 

All done
                                     CVXPY                                     
                                     v1.3.2                                    
(CVXPY) Dec 03 03:35:29 PM: Your problem has 220 variables, 26224 constraints, and 0 parameters.
(CVXPY) Dec 03 03:35:33 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Dec 03 03:35:33 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Dec 03 03:35:33 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
--------------------------------------------

/home/onoue/ws/lukasiewicz_1/myenv/lib/python3.10/site-packages/cvxpy/problems/problem.py:1387: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
/home/onoue/ws/lukasiewicz_1/myenv/lib/python3.10/site-packages/cvxpy/atoms/elementwise/log.py:35: RuntimeWarning: invalid value encountered in log
  return np.log(values[0])



学習時間: 1986.968360900879 秒





,Pregnancies,Glucose,BloodPressure,SkinThickness,BMI,DiabetesPedigreeFunction,Age,target
419,3,129,64,29,26.4,0.219,28,1
186,8,181,68,36,30.1,0.615,60,1
556,1,97,70,40,38.1,0.218,30,-1
738,2,99,60,17,36.6,0.453,21,-1
320,4,129,60,12,27.5,0.527,31,-1


Outcome's coefficients: 
[ 0.02117143  0.00467079 -0.00795573  0.00042139  0.00412311  0.09864862
  0.00285028  0.0025941 ]

Accuracy: 
0.7388059701492538

Confusion matrix: 
[[78 12]
 [23 21]]

classification report: 
              precision    recall  f1-score   support

        -1.0       0.77      0.87      0.82        90
         1.0       0.64      0.48      0.55        44

    accuracy                           0.74       134
   macro avg       0.70      0.67      0.68       134
weighted avg       0.73      0.74      0.73       134


auc: 
0.7974747474747474


In [82]:
tmp_vals = cp.Variable(3)
tmp = np.random.rand(3)
tmptmp = cp.multiply(tmp_vals, tmp)
print(tmptmp.shape)

(3,)


In [66]:
from src.setup_problem_loss_custom_obj import Setup

data_dir_path = os.path.join(project_dir_path, 'inputs/pima_indian_diabetes')
train_data_dir_path = os.path.join(data_dir_path, "train_4")
file_list = os.listdir(train_data_dir_path)

L_files = [filename.split('.csv')[0] for filename in file_list 
           if filename.startswith('L') and filename.endswith('.csv')]

U_files = [filename.split('.csv')[0] for filename in file_list 
           if filename.startswith('U') and filename.endswith('.csv')]

file_names_dict = {
    'supervised': L_files,
    'unsupervised': U_files,
    'rule': ['rules_4']
}

c2_dict = {1: 10, 2: 2.5}

obj_constructor = construct_objective_function
problem_instance = Setup(train_data_dir_path, file_names_dict, obj_constructor)
objective, constraints = problem_instance.main(c1=10, c2_dict=c2_dict)

start_time = time.time()
problem = cp.Problem(objective, constraints)
result = problem.solve(verbose=True)
end_time = time.time()
print()
print(f'学習時間: {end_time - start_time} 秒')


print()
print()
print()


test_data_dir_path = os.path.join(data_dir_path, "test_4")
file_path = os.path.join(test_data_dir_path, "L_Outcome.csv")
test_df = pd.read_csv(file_path, index_col=0)
display(test_df.head())

test_data = {
    'Outcome': np.array(test_df)
}

p_dict = problem_instance.predicates_dict
selected_predicates = ['Outcome']
selected_p_dict = {key: value for key, value in p_dict.items() if key in selected_predicates}

X_test = test_data['Outcome'][:, :-1]
y_test = test_data['Outcome'][:, -1]

y_pred = p_dict['Outcome'](X_test).value

y_pred_interpreted = np.where(y_pred >= 0.5, 1, -1)

print("Outcome's coefficients: ")
print(problem_instance.w_j.value[3])
print()

accuracy = accuracy_score(y_test, y_pred_interpreted)
print("Accuracy: ")
print(accuracy)
print()

conf_matrix = confusion_matrix(y_test, y_pred_interpreted)
print("Confusion matrix: ")
print(conf_matrix)
print()

class_report = classification_report(y_test, y_pred_interpreted)
print("classification report: ")
print(class_report)
print()

roc_auc = roc_auc_score(y_test, y_pred)
print("auc: ")
print(roc_auc)

Loading data ...
Done in 0.6158018112182617 seconds! 

Loading rules ...
Done in 0.014562606811523438 seconds! 

Identifying predicates ...
Done in 0.006165504455566406 seconds! 

Constructing objective function ...
c1: 10
c2: 10
c2: 2.5
Done in 0.09418749809265137 seconds! 

Constructing constraints ...
Done in 22.27830958366394 seconds! 

All done
                                     CVXPY                                     
                                     v1.3.2                                    
(CVXPY) Dec 03 03:04:03 PM: Your problem has 220 variables, 26224 constraints, and 0 parameters.
(CVXPY) Dec 03 03:04:07 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Dec 03 03:04:07 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Dec 03 03:04:07 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-----------------------------------------------

/home/onoue/ws/lukasiewicz_1/myenv/lib/python3.10/site-packages/cvxpy/problems/problem.py:1387: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
/home/onoue/ws/lukasiewicz_1/myenv/lib/python3.10/site-packages/cvxpy/atoms/elementwise/log.py:35: RuntimeWarning: invalid value encountered in log
  return np.log(values[0])


,Pregnancies,Glucose,BloodPressure,SkinThickness,BMI,DiabetesPedigreeFunction,Age,target
419,3,129,64,29,26.4,0.219,28,1
186,8,181,68,36,30.1,0.615,60,1
556,1,97,70,40,38.1,0.218,30,-1
738,2,99,60,17,36.6,0.453,21,-1
320,4,129,60,12,27.5,0.527,31,-1


Outcome's coefficients: 
[ 2.36377876e-02  4.54547797e-03 -7.26421152e-03  3.19126050e-05
  3.82009736e-03  1.16600680e-01  1.76903795e-03  5.46321907e-03]

Accuracy: 
0.7611940298507462

Confusion matrix: 
[[79 11]
 [21 23]]

classification report: 
              precision    recall  f1-score   support

        -1.0       0.79      0.88      0.83        90
         1.0       0.68      0.52      0.59        44

    accuracy                           0.76       134
   macro avg       0.73      0.70      0.71       134
weighted avg       0.75      0.76      0.75       134


auc: 
0.8035353535353535


In [69]:
from src.setup_problem_loss_custom_obj import Setup

data_dir_path = os.path.join(project_dir_path, 'inputs/pima_indian_diabetes')
train_data_dir_path = os.path.join(data_dir_path, "train_4")
file_list = os.listdir(train_data_dir_path)

L_files = [filename.split('.csv')[0] for filename in file_list 
           if filename.startswith('L') and filename.endswith('.csv')]

U_files = [filename.split('.csv')[0] for filename in file_list 
           if filename.startswith('U') and filename.endswith('.csv')]

file_names_dict = {
    'supervised': L_files,
    'unsupervised': U_files,
    'rule': ['rules_4']
}

c2_dict = {1: 0.001, 2: 1}

obj_constructor = construct_objective_function
problem_instance = Setup(train_data_dir_path, file_names_dict, obj_constructor)
objective, constraints = problem_instance.main(c1=10, c2_dict=c2_dict)

start_time = time.time()
problem = cp.Problem(objective, constraints)
result = problem.solve(verbose=True)
end_time = time.time()
print()
print(f'学習時間: {end_time - start_time} 秒')


print()
print()
print()


test_data_dir_path = os.path.join(data_dir_path, "test_4")
file_path = os.path.join(test_data_dir_path, "L_Outcome.csv")
test_df = pd.read_csv(file_path, index_col=0)
display(test_df.head())

test_data = {
    'Outcome': np.array(test_df)
}

p_dict = problem_instance.predicates_dict
selected_predicates = ['Outcome']
selected_p_dict = {key: value for key, value in p_dict.items() if key in selected_predicates}

X_test = test_data['Outcome'][:, :-1]
y_test = test_data['Outcome'][:, -1]

y_pred = p_dict['Outcome'](X_test).value

y_pred_interpreted = np.where(y_pred >= 0.5, 1, -1)

print("Outcome's coefficients: ")
print(problem_instance.w_j.value[3])
print()

accuracy = accuracy_score(y_test, y_pred_interpreted)
print("Accuracy: ")
print(accuracy)
print()

conf_matrix = confusion_matrix(y_test, y_pred_interpreted)
print("Confusion matrix: ")
print(conf_matrix)
print()

class_report = classification_report(y_test, y_pred_interpreted)
print("classification report: ")
print(class_report)
print()

roc_auc = roc_auc_score(y_test, y_pred)
print("auc: ")
print(roc_auc)

Loading data ...
Done in 0.17029690742492676 seconds! 

Loading rules ...
Done in 0.0029404163360595703 seconds! 

Identifying predicates ...
Done in 0.00299835205078125 seconds! 

Constructing objective function ...
c1: 10
c2: 0.001
c2: 1
Done in 0.04529714584350586 seconds! 

Constructing constraints ...
Done in 15.918298721313477 seconds! 

All done
                                     CVXPY                                     
                                     v1.3.2                                    
(CVXPY) Dec 03 03:06:52 PM: Your problem has 220 variables, 26224 constraints, and 0 parameters.
(CVXPY) Dec 03 03:06:56 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Dec 03 03:06:56 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Dec 03 03:06:56 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
--------------------------------------------

/home/onoue/ws/lukasiewicz_1/myenv/lib/python3.10/site-packages/cvxpy/atoms/elementwise/log.py:35: RuntimeWarning: invalid value encountered in log
  return np.log(values[0])


,Pregnancies,Glucose,BloodPressure,SkinThickness,BMI,DiabetesPedigreeFunction,Age,target
419,3,129,64,29,26.4,0.219,28,1
186,8,181,68,36,30.1,0.615,60,1
556,1,97,70,40,38.1,0.218,30,-1
738,2,99,60,17,36.6,0.453,21,-1
320,4,129,60,12,27.5,0.527,31,-1


Outcome's coefficients: 
[ 2.36537275e-02  4.54761265e-03 -7.26466801e-03  3.56377888e-05
  3.82447834e-03  1.15691214e-01  1.76522925e-03  5.46380205e-03]

Accuracy: 
0.7611940298507462

Confusion matrix: 
[[79 11]
 [21 23]]

classification report: 
              precision    recall  f1-score   support

        -1.0       0.79      0.88      0.83        90
         1.0       0.68      0.52      0.59        44

    accuracy                           0.76       134
   macro avg       0.73      0.70      0.71       134
weighted avg       0.75      0.76      0.75       134


auc: 
0.8032828282828283


In [56]:
from src.setup_problem_loss_custom_obj import Setup

data_dir_path = os.path.join(project_dir_path, 'inputs/pima_indian_diabetes')
train_data_dir_path = os.path.join(data_dir_path, "train_4")
file_list = os.listdir(train_data_dir_path)

L_files = [filename.split('.csv')[0] for filename in file_list 
           if filename.startswith('L') and filename.endswith('.csv')]

U_files = [filename.split('.csv')[0] for filename in file_list 
           if filename.startswith('U') and filename.endswith('.csv')]

file_names_dict = {
    'supervised': L_files,
    'unsupervised': U_files,
    'rule': ['rules_4']
}

c2_dict = {1: 0.1, 2: 0.1}

obj_constructor = construct_objective_function
problem_instance = Setup(train_data_dir_path, file_names_dict, obj_constructor)
objective, constraints = problem_instance.main(c1=10, c2_dict=c2_dict)

start_time = time.time()
problem = cp.Problem(objective, constraints)
result = problem.solve(verbose=True)
end_time = time.time()
print()
print(f'学習時間: {end_time - start_time} 秒')


print()
print()
print()


test_data_dir_path = os.path.join(data_dir_path, "test_4")
file_path = os.path.join(test_data_dir_path, "L_Outcome.csv")
test_df = pd.read_csv(file_path, index_col=0)
display(test_df.head())

test_data = {
    'Outcome': np.array(test_df)
}

p_dict = problem_instance.predicates_dict
selected_predicates = ['Outcome']
selected_p_dict = {key: value for key, value in p_dict.items() if key in selected_predicates}

X_test = test_data['Outcome'][:, :-1]
y_test = test_data['Outcome'][:, -1]

y_pred = p_dict['Outcome'](X_test).value

y_pred_interpreted = np.where(y_pred >= 0.5, 1, -1)

print("Outcome's coefficients: ")
print(problem_instance.w_j.value[3])
print()

accuracy = accuracy_score(y_test, y_pred_interpreted)
print("Accuracy: ")
print(accuracy)
print()

conf_matrix = confusion_matrix(y_test, y_pred_interpreted)
print("Confusion matrix: ")
print(conf_matrix)
print()

class_report = classification_report(y_test, y_pred_interpreted)
print("classification report: ")
print(class_report)
print()

roc_auc = roc_auc_score(y_test, y_pred)
print("auc: ")
print(roc_auc)

Loading data ...
Done in 0.17425179481506348 seconds! 

Loading rules ...
Done in 0.003795146942138672 seconds! 

Identifying predicates ...
Done in 0.0027365684509277344 seconds! 

Constructing objective function ...


c1: 10
c2: 0.1
c2: 0.1
Done in 0.0784158706665039 seconds! 

Constructing constraints ...
Done in 18.691462993621826 seconds! 

All done
                                     CVXPY                                     
                                     v1.3.2                                    
(CVXPY) Dec 03 01:56:13 PM: Your problem has 220 variables, 26224 constraints, and 0 parameters.
(CVXPY) Dec 03 01:56:17 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Dec 03 01:56:17 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Dec 03 01:56:17 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Dec 03 01:56:2

/home/onoue/ws/lukasiewicz_1/myenv/lib/python3.10/site-packages/cvxpy/atoms/elementwise/log.py:35: RuntimeWarning: invalid value encountered in log
  return np.log(values[0])


,Pregnancies,Glucose,BloodPressure,SkinThickness,BMI,DiabetesPedigreeFunction,Age,target
419,3,129,64,29,26.4,0.219,28,1
186,8,181,68,36,30.1,0.615,60,1
556,1,97,70,40,38.1,0.218,30,-1
738,2,99,60,17,36.6,0.453,21,-1
320,4,129,60,12,27.5,0.527,31,-1


Outcome's coefficients: 
[ 2.36537290e-02  4.54761302e-03 -7.26466902e-03  3.56390934e-05
  3.82447929e-03  1.15691092e-01  1.76522959e-03  5.46380095e-03]

Accuracy: 
0.7611940298507462

Confusion matrix: 
[[79 11]
 [21 23]]

classification report: 
              precision    recall  f1-score   support

        -1.0       0.79      0.88      0.83        90
         1.0       0.68      0.52      0.59        44

    accuracy                           0.76       134
   macro avg       0.73      0.70      0.71       134
weighted avg       0.75      0.76      0.75       134


auc: 
0.8032828282828283


In [57]:
from src.setup_problem_loss_custom_obj import Setup

data_dir_path = os.path.join(project_dir_path, 'inputs/pima_indian_diabetes')
train_data_dir_path = os.path.join(data_dir_path, "train_4")
file_list = os.listdir(train_data_dir_path)

L_files = [filename.split('.csv')[0] for filename in file_list 
           if filename.startswith('L') and filename.endswith('.csv')]

U_files = [filename.split('.csv')[0] for filename in file_list 
           if filename.startswith('U') and filename.endswith('.csv')]

file_names_dict = {
    'supervised': L_files,
    'unsupervised': U_files,
    'rule': ['rules_4']
}

c2_dict = {1: 10, 2: 10}

obj_constructor = construct_objective_function
problem_instance = Setup(train_data_dir_path, file_names_dict, obj_constructor)
objective, constraints = problem_instance.main(c1=0.01, c2_dict=c2_dict)

start_time = time.time()
problem = cp.Problem(objective, constraints)
result = problem.solve(verbose=True)
end_time = time.time()
print()
print(f'学習時間: {end_time - start_time} 秒')


print()
print()
print()


test_data_dir_path = os.path.join(data_dir_path, "test_4")
file_path = os.path.join(test_data_dir_path, "L_Outcome.csv")
test_df = pd.read_csv(file_path, index_col=0)
display(test_df.head())

test_data = {
    'Outcome': np.array(test_df)
}

p_dict = problem_instance.predicates_dict
selected_predicates = ['Outcome']
selected_p_dict = {key: value for key, value in p_dict.items() if key in selected_predicates}

X_test = test_data['Outcome'][:, :-1]
y_test = test_data['Outcome'][:, -1]

y_pred = p_dict['Outcome'](X_test).value

y_pred_interpreted = np.where(y_pred >= 0.5, 1, -1)

print("Outcome's coefficients: ")
print(problem_instance.w_j.value[3])
print()

accuracy = accuracy_score(y_test, y_pred_interpreted)
print("Accuracy: ")
print(accuracy)
print()

conf_matrix = confusion_matrix(y_test, y_pred_interpreted)
print("Confusion matrix: ")
print(conf_matrix)
print()

class_report = classification_report(y_test, y_pred_interpreted)
print("classification report: ")
print(class_report)
print()

roc_auc = roc_auc_score(y_test, y_pred)
print("auc: ")
print(roc_auc)

Loading data ...
Done in 0.1699657440185547 seconds! 

Loading rules ...
Done in 0.004059314727783203 seconds! 

Identifying predicates ...
Done in 0.01209259033203125 seconds! 

Constructing objective function ...


c1: 0.01
c2: 10
c2: 10
Done in 0.07747530937194824 seconds! 

Constructing constraints ...
Done in 12.787901878356934 seconds! 

All done
                                     CVXPY                                     
                                     v1.3.2                                    
(CVXPY) Dec 03 01:58:14 PM: Your problem has 220 variables, 26224 constraints, and 0 parameters.
(CVXPY) Dec 03 01:58:18 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Dec 03 01:58:18 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Dec 03 01:58:18 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Dec 03 01:58:

/home/onoue/ws/lukasiewicz_1/myenv/lib/python3.10/site-packages/cvxpy/problems/problem.py:1387: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
/home/onoue/ws/lukasiewicz_1/myenv/lib/python3.10/site-packages/cvxpy/atoms/elementwise/log.py:35: RuntimeWarning: invalid value encountered in log
  return np.log(values[0])


,Pregnancies,Glucose,BloodPressure,SkinThickness,BMI,DiabetesPedigreeFunction,Age,target
419,3,129,64,29,26.4,0.219,28,1
186,8,181,68,36,30.1,0.615,60,1
556,1,97,70,40,38.1,0.218,30,-1
738,2,99,60,17,36.6,0.453,21,-1
320,4,129,60,12,27.5,0.527,31,-1


Outcome's coefficients: 
[ 0.0237284   0.00462123 -0.00726112  0.0001971   0.00401843  0.06698493
  0.00179667  0.00526735]

Accuracy: 
0.7686567164179104

Confusion matrix: 
[[80 10]
 [21 23]]

classification report: 
              precision    recall  f1-score   support

        -1.0       0.79      0.89      0.84        90
         1.0       0.70      0.52      0.60        44

    accuracy                           0.77       134
   macro avg       0.74      0.71      0.72       134
weighted avg       0.76      0.77      0.76       134


auc: 
0.8022727272727272


In [59]:
from src.setup_problem_loss_custom_obj import Setup

data_dir_path = os.path.join(project_dir_path, 'inputs/pima_indian_diabetes')
train_data_dir_path = os.path.join(data_dir_path, "train_4")
file_list = os.listdir(train_data_dir_path)

L_files = [filename.split('.csv')[0] for filename in file_list 
           if filename.startswith('L') and filename.endswith('.csv')]

U_files = [filename.split('.csv')[0] for filename in file_list 
           if filename.startswith('U') and filename.endswith('.csv')]

file_names_dict = {
    'supervised': L_files,
    'unsupervised': U_files,
    'rule': ['rules_4']
}

c2_dict = {1: 10, 2: 0.1}

obj_constructor = construct_objective_function
problem_instance = Setup(train_data_dir_path, file_names_dict, obj_constructor)
objective, constraints = problem_instance.main(c1=0.1, c2_dict=c2_dict)

start_time = time.time()
problem = cp.Problem(objective, constraints)
result = problem.solve(verbose=True)
end_time = time.time()
print()
print(f'学習時間: {end_time - start_time} 秒')


print()
print()
print()


test_data_dir_path = os.path.join(data_dir_path, "test_4")
file_path = os.path.join(test_data_dir_path, "L_Outcome.csv")
test_df = pd.read_csv(file_path, index_col=0)
display(test_df.head())

test_data = {
    'Outcome': np.array(test_df)
}

p_dict = problem_instance.predicates_dict
selected_predicates = ['Outcome']
selected_p_dict = {key: value for key, value in p_dict.items() if key in selected_predicates}

X_test = test_data['Outcome'][:, :-1]
y_test = test_data['Outcome'][:, -1]

y_pred = p_dict['Outcome'](X_test).value

y_pred_interpreted = np.where(y_pred >= 0.5, 1, -1)

print("Outcome's coefficients: ")
print(problem_instance.w_j.value[3])
print()

accuracy = accuracy_score(y_test, y_pred_interpreted)
print("Accuracy: ")
print(accuracy)
print()

conf_matrix = confusion_matrix(y_test, y_pred_interpreted)
print("Confusion matrix: ")
print(conf_matrix)
print()

class_report = classification_report(y_test, y_pred_interpreted)
print("classification report: ")
print(class_report)
print()

roc_auc = roc_auc_score(y_test, y_pred)
print("auc: ")
print(roc_auc)

Loading data ...
Done in 0.21732497215270996 seconds! 

Loading rules ...
Done in 0.0038919448852539062 seconds! 

Identifying predicates ...
Done in 0.0024127960205078125 seconds! 

Constructing objective function ...
c1: 0.1
c2: 10
c2: 0.1
Done in 0.036238670349121094 seconds! 

Constructing constraints ...
Done in 14.69363284111023 seconds! 

All done
                                     CVXPY                                     
                                     v1.3.2                                    
(CVXPY) Dec 03 02:04:33 PM: Your problem has 220 variables, 26224 constraints, and 0 parameters.
(CVXPY) Dec 03 02:04:37 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Dec 03 02:04:37 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Dec 03 02:04:37 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
------------------------------------------

/home/onoue/ws/lukasiewicz_1/myenv/lib/python3.10/site-packages/cvxpy/atoms/elementwise/log.py:35: RuntimeWarning: invalid value encountered in log
  return np.log(values[0])


,Pregnancies,Glucose,BloodPressure,SkinThickness,BMI,DiabetesPedigreeFunction,Age,target
419,3,129,64,29,26.4,0.219,28,1
186,8,181,68,36,30.1,0.615,60,1
556,1,97,70,40,38.1,0.218,30,-1
738,2,99,60,17,36.6,0.453,21,-1
320,4,129,60,12,27.5,0.527,31,-1


Outcome's coefficients: 
[ 2.36870105e-02  4.56718946e-03 -7.26142941e-03  6.70780629e-05
  3.83557616e-03  1.08229571e-01  1.74358851e-03  5.45076281e-03]

Accuracy: 
0.7611940298507462

Confusion matrix: 
[[79 11]
 [21 23]]

classification report: 
              precision    recall  f1-score   support

        -1.0       0.79      0.88      0.83        90
         1.0       0.68      0.52      0.59        44

    accuracy                           0.76       134
   macro avg       0.73      0.70      0.71       134
weighted avg       0.75      0.76      0.75       134


auc: 
0.8037878787878787


In [60]:
from src.setup_problem_loss_custom_obj import Setup

data_dir_path = os.path.join(project_dir_path, 'inputs/pima_indian_diabetes')
train_data_dir_path = os.path.join(data_dir_path, "train_4")
file_list = os.listdir(train_data_dir_path)

L_files = [filename.split('.csv')[0] for filename in file_list 
           if filename.startswith('L') and filename.endswith('.csv')]

U_files = [filename.split('.csv')[0] for filename in file_list 
           if filename.startswith('U') and filename.endswith('.csv')]

file_names_dict = {
    'supervised': L_files,
    'unsupervised': U_files,
    'rule': ['rules_4']
}

c2_dict = {1: 0.1, 2: 10}

obj_constructor = construct_objective_function
problem_instance = Setup(train_data_dir_path, file_names_dict, obj_constructor)
objective, constraints = problem_instance.main(c1=0.1, c2_dict=c2_dict)

start_time = time.time()
problem = cp.Problem(objective, constraints)
result = problem.solve(verbose=True)
end_time = time.time()
print()
print(f'学習時間: {end_time - start_time} 秒')


print()
print()
print()


test_data_dir_path = os.path.join(data_dir_path, "test_4")
file_path = os.path.join(test_data_dir_path, "L_Outcome.csv")
test_df = pd.read_csv(file_path, index_col=0)
display(test_df.head())

test_data = {
    'Outcome': np.array(test_df)
}

p_dict = problem_instance.predicates_dict
selected_predicates = ['Outcome']
selected_p_dict = {key: value for key, value in p_dict.items() if key in selected_predicates}

X_test = test_data['Outcome'][:, :-1]
y_test = test_data['Outcome'][:, -1]

y_pred = p_dict['Outcome'](X_test).value

y_pred_interpreted = np.where(y_pred >= 0.5, 1, -1)

print("Outcome's coefficients: ")
print(problem_instance.w_j.value[3])
print()

accuracy = accuracy_score(y_test, y_pred_interpreted)
print("Accuracy: ")
print(accuracy)
print()

conf_matrix = confusion_matrix(y_test, y_pred_interpreted)
print("Confusion matrix: ")
print(conf_matrix)
print()

class_report = classification_report(y_test, y_pred_interpreted)
print("classification report: ")
print(class_report)
print()

roc_auc = roc_auc_score(y_test, y_pred)
print("auc: ")
print(roc_auc)

Loading data ...
Done in 0.25171589851379395 seconds! 

Loading rules ...
Done in 0.004272937774658203 seconds! 

Identifying predicates ...
Done in 0.002713441848754883 seconds! 

Constructing objective function ...
c1: 0.1
c2: 0.1
c2: 10
Done in 0.09728407859802246 seconds! 

Constructing constraints ...
Done in 14.231910705566406 seconds! 

All done
                                     CVXPY                                     
                                     v1.3.2                                    
(CVXPY) Dec 03 02:06:48 PM: Your problem has 220 variables, 26224 constraints, and 0 parameters.
(CVXPY) Dec 03 02:06:52 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Dec 03 02:06:52 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Dec 03 02:06:52 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
--------------------------------------------

/home/onoue/ws/lukasiewicz_1/myenv/lib/python3.10/site-packages/cvxpy/atoms/elementwise/log.py:35: RuntimeWarning: invalid value encountered in log
  return np.log(values[0])


,Pregnancies,Glucose,BloodPressure,SkinThickness,BMI,DiabetesPedigreeFunction,Age,target
419,3,129,64,29,26.4,0.219,28,1
186,8,181,68,36,30.1,0.615,60,1
556,1,97,70,40,38.1,0.218,30,-1
738,2,99,60,17,36.6,0.453,21,-1
320,4,129,60,12,27.5,0.527,31,-1


Outcome's coefficients: 
[ 2.36870105e-02  4.56718946e-03 -7.26142941e-03  6.70780629e-05
  3.83557616e-03  1.08229571e-01  1.74358851e-03  5.45076281e-03]

Accuracy: 
0.7611940298507462

Confusion matrix: 
[[79 11]
 [21 23]]

classification report: 
              precision    recall  f1-score   support

        -1.0       0.79      0.88      0.83        90
         1.0       0.68      0.52      0.59        44

    accuracy                           0.76       134
   macro avg       0.73      0.70      0.71       134
weighted avg       0.75      0.76      0.75       134


auc: 
0.8037878787878787


In [63]:
from src.setup_problem_loss_custom_obj import Setup

data_dir_path = os.path.join(project_dir_path, 'inputs/pima_indian_diabetes')
train_data_dir_path = os.path.join(data_dir_path, "train_4")
file_list = os.listdir(train_data_dir_path)

L_files = [filename.split('.csv')[0] for filename in file_list 
           if filename.startswith('L') and filename.endswith('.csv')]

U_files = [filename.split('.csv')[0] for filename in file_list 
           if filename.startswith('U') and filename.endswith('.csv')]

file_names_dict = {
    'supervised': L_files,
    'unsupervised': U_files,
    'rule': ['rules_4']
}

c2_dict = {1: 1000, 2: 1000}

obj_constructor = construct_objective_function
problem_instance = Setup(train_data_dir_path, file_names_dict, obj_constructor)
objective, constraints = problem_instance.main(c1=0.001, c2_dict=c2_dict)

start_time = time.time()
problem = cp.Problem(objective, constraints)
result = problem.solve(verbose=True)
end_time = time.time()
print()
print(f'学習時間: {end_time - start_time} 秒')


print()
print()
print()


test_data_dir_path = os.path.join(data_dir_path, "test_4")
file_path = os.path.join(test_data_dir_path, "L_Outcome.csv")
test_df = pd.read_csv(file_path, index_col=0)
display(test_df.head())

test_data = {
    'Outcome': np.array(test_df)
}

p_dict = problem_instance.predicates_dict
selected_predicates = ['Outcome']
selected_p_dict = {key: value for key, value in p_dict.items() if key in selected_predicates}

X_test = test_data['Outcome'][:, :-1]
y_test = test_data['Outcome'][:, -1]

y_pred = p_dict['Outcome'](X_test).value

y_pred_interpreted = np.where(y_pred >= 0.5, 1, -1)

print("Outcome's coefficients: ")
print(problem_instance.w_j.value[3])
print()

accuracy = accuracy_score(y_test, y_pred_interpreted)
print("Accuracy: ")
print(accuracy)
print()

conf_matrix = confusion_matrix(y_test, y_pred_interpreted)
print("Confusion matrix: ")
print(conf_matrix)
print()

class_report = classification_report(y_test, y_pred_interpreted)
print("classification report: ")
print(class_report)
print()

roc_auc = roc_auc_score(y_test, y_pred)
print("auc: ")
print(roc_auc)

Loading data ...
Done in 0.23201775550842285 seconds! 

Loading rules ...
Done in 0.0031595230102539062 seconds! 

Identifying predicates ...
Done in 0.002337932586669922 seconds! 

Constructing objective function ...
c1: 0.001
c2: 1000
c2: 1000
Done in 0.0710763931274414 seconds! 

Constructing constraints ...
Done in 16.208917140960693 seconds! 

All done
                                     CVXPY                                     
                                     v1.3.2                                    
(CVXPY) Dec 03 02:18:58 PM: Your problem has 220 variables, 26224 constraints, and 0 parameters.
(CVXPY) Dec 03 02:19:01 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Dec 03 02:19:01 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Dec 03 02:19:01 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
---------------------------------------

/home/onoue/ws/lukasiewicz_1/myenv/lib/python3.10/site-packages/cvxpy/problems/problem.py:1387: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
/home/onoue/ws/lukasiewicz_1/myenv/lib/python3.10/site-packages/cvxpy/atoms/elementwise/log.py:35: RuntimeWarning: invalid value encountered in log
  return np.log(values[0])


,Pregnancies,Glucose,BloodPressure,SkinThickness,BMI,DiabetesPedigreeFunction,Age,target
419,3,129,64,29,26.4,0.219,28,1
186,8,181,68,36,30.1,0.615,60,1
556,1,97,70,40,38.1,0.218,30,-1
738,2,99,60,17,36.6,0.453,21,-1
320,4,129,60,12,27.5,0.527,31,-1


Outcome's coefficients: 
[ 0.02251493  0.00465483 -0.00725829  0.00026224  0.00437261  0.01506199
  0.0020752   0.00492085]

Accuracy: 
0.7611940298507462

Confusion matrix: 
[[79 11]
 [21 23]]

classification report: 
              precision    recall  f1-score   support

        -1.0       0.79      0.88      0.83        90
         1.0       0.68      0.52      0.59        44

    accuracy                           0.76       134
   macro avg       0.73      0.70      0.71       134
weighted avg       0.75      0.76      0.75       134


auc: 
0.7977272727272727


In [ ]:
[ 2.36537290e-02  4.54761302e-03 -7.26466902e-03  3.56390934e-05
  3.82447929e-03  1.15691092e-01  1.76522959e-03  5.46380095e-03]
[ 2.36537290e-02  4.54761302e-03 -7.26466902e-03  3.56390934e-05
  3.82447929e-03  1.15691092e-01  1.76522959e-03  5.46380095e-03]
[ 2.36537290e-02  4.54761302e-03 -7.26466902e-03  3.56390934e-05
  3.82447929e-03  1.15691092e-01  1.76522959e-03  5.46380095e-03]


[ 2.36870105e-02  4.56718946e-03 -7.26142941e-03  6.70780629e-05
  3.83557616e-03  1.08229571e-01  1.74358851e-03  5.45076281e-03]
[ 2.36870105e-02  4.56718946e-03 -7.26142941e-03  6.70780629e-05
  3.83557616e-03  1.08229571e-01  1.74358851e-03  5.45076281e-03]

In [ ]:
[ 0.02251493  0.00465483 -0.00725829  0.00026224  0.00437261  0.01506199
  0.0020752   0.00492085]

In [ ]:
[ 0.02251493  0.00465483 -0.00725829  0.00026224  0.00437261  0.01506199
  0.0020752   0.00492085]